# Limpieza de datos
Como punto inicial se importan los paquetes necesarios para el procesamiento de los datos

In [65]:
import numpy as np
import pandas as pd

Ahora se crea un data frame usando la función read_csv de pandas

In [66]:
df = pd.read_csv('InfoNeza/Delitos/Delitos.csv')
print(df)

          ID           HORA      FECHA         MES                     DELITO  \
0          1  22:30 A 04:00   1/1/2016       ENERO           ROBO DE VEHICULO   
1          2           6:40   1/1/2016       ENERO           ROBO DE VEHICULO   
2          3          13:30   1/1/2016       ENERO           ROBO DE VEHICULO   
3          4  03:00 A 05:00   1/1/2016       ENERO           ROBO DE VEHICULO   
4          5          14:00   1/1/2016       ENERO           ROBO DE VEHICULO   
5          6           0:05   1/1/2016       ENERO                  SECUESTRO   
6          7           4:00   1/1/2016       ENERO     ROBO A CASA HABITACION   
7          8           4:58   1/1/2016       ENERO     ROBO A CASA HABITACION   
8          9          18:13   1/1/2016       ENERO     ROBO A CASA HABITACION   
9         10          19:40   1/1/2016       ENERO     ROBO A CASA HABITACION   
10        11          21:10   1/1/2016       ENERO     ROBO A CASA HABITACION   
11        12           7:20 

Una vez cargados los datos en el data frame es posible acceder a los datos de cada columna usando el nombre correspondiente. Para ver las columnas contenidas en el dataframe basta con usar el atributo columns:

In [67]:
print(df.columns)

Index(['ID', 'HORA', 'FECHA', 'MES', 'DELITO', 'CALLE', 'No.', 'ENTRE_CALLE_1',
       'ENTRE_CALLE_2', 'COLONIA', 'SECTOR', 'CUADRANTE', 'TURNO', 'X', 'Y'],
      dtype='object')


De acuerdo a lo anterior, para imprimir la calle en la que fueron cometidos los primeros 10 delitos se puede utilizar ademas el método head().

In [68]:
print(df['CALLE'].head(10))

0      VALLE DE MEXICO
1        RANCHO GRANDE
2             LAURELES
3        LAGO GRAN OSO
4        LA CHAPARRITA
5               FAISAN
6      VALLE DE MEXICO
7        AV. PANTITLAN
8            CLAVELERO
9    VALLE DE IRAGUADI
Name: CALLE, dtype: object


De manera similar se puede utilizar el metodo tail() para imprimir los datos al final de la columna:

In [69]:
print(df['No.'].tail(20))

16405           NaN
16406           NaN
16407           NaN
16408           NaN
16409           NaN
16410           NaN
16411           NaN
16412           NaN
16413           NaN
16414           NaN
16415           NaN
16416           NaN
16417           154
16418    MZ.30 LT.8
16419           NaN
16420            48
16421           NaN
16422           NaN
16423           NaN
16424           NaN
Name: No., dtype: object


## Rellenar campos vacíos

Como puede observarse varios de los registros de la columna "No." no tienen un valor asociado. Una de las labores principales de cualquier proceso de limpieza de datos es asegurarse que los mismos estén estandarizados, en consecuencia es necesario eliminar esta anomalia. Para tal fin, cada registro que no tenga un valor en la columna "No." serán llenado con la leyenda "Sin No.". Para realizar esta transformación basta con usar el metodo fillna() de una serie. Este método regresa una copia de la serie donde cada NaN es sustituido por el valor dado.

In [70]:
print(df['No.'].fillna('Sin No.').tail(20))

16405       Sin No.
16406       Sin No.
16407       Sin No.
16408       Sin No.
16409       Sin No.
16410       Sin No.
16411       Sin No.
16412       Sin No.
16413       Sin No.
16414       Sin No.
16415       Sin No.
16416       Sin No.
16417           154
16418    MZ.30 LT.8
16419       Sin No.
16420            48
16421       Sin No.
16422       Sin No.
16423       Sin No.
16424       Sin No.
Name: No., dtype: object


Ya que la columna 'No.' no es la única columna que presenta esta anomalia, es necesario realizar el proceso anterior para todas las columnas del dataframe.

In [71]:
for column in df.columns:
    df[column] = df[column].fillna('Sin ' + column)

El proceso anterior elimina las anomalias relacionadas con datos faltantes. Sin embargo, existen otros problemas que deben ser solucionados antes de que los datos puedan considerarse estandarizados. Por ejemplo, considere los datos que muestran la hora en la que fue cometido un delito.

In [72]:
print(df['HORA'].head(10))

0    22:30 A 04:00
1             6:40
2            13:30
3    03:00 A 05:00
4            14:00
5             0:05
6             4:00
7             4:58
8            18:13
9            19:40
Name: HORA, dtype: object


## Unificar el formato de las horas

Como se puede observar los datos tienen formatos diferentes. Existen registros que contienen una hora de inicio y una hora de fin. Con el objetivo de tener registro con entradas consistentes los datos de esta columna deben ser divididos. La columna 'HORA_INICIO' tendrá la hora en la que comenzó el delito, y la columna 'HORA_FIN' tendrá la hora en la que terminó el delito. Para aquellos registros que solo tengan una hora se tendra que la hora de inicio y la hora de fin será la misma.

In [73]:
df['HORA_INICIO'], df['HORA_FIN'] = df['HORA'].str.split(' A ', 1).str
df['HORA_FIN'] = np.where(pd.isna(df['HORA_FIN']), df['HORA_INICIO'], df['HORA_FIN'])

## Eliminar espacios al inicio y final de la cadena de caracteres

In [74]:
for column in df.columns:
    df[column] = df[column].map(lambda x: str(x).strip())

## Cambiar el orden de las columnas

Para modificar el orden de las columnas agregadas recientemente y tenerlas de forma organizada, se puede hacer de la siguiente forma.

In [75]:
cols = df.columns.tolist()
cols

['ID',
 'HORA',
 'FECHA',
 'MES',
 'DELITO',
 'CALLE',
 'No.',
 'ENTRE_CALLE_1',
 'ENTRE_CALLE_2',
 'COLONIA',
 'SECTOR',
 'CUADRANTE',
 'TURNO',
 'X',
 'Y',
 'HORA_INICIO',
 'HORA_FIN']

In [76]:
cols = cols[:1] + cols[-2:] + cols[2:-2:]
cols

['ID',
 'HORA_INICIO',
 'HORA_FIN',
 'FECHA',
 'MES',
 'DELITO',
 'CALLE',
 'No.',
 'ENTRE_CALLE_1',
 'ENTRE_CALLE_2',
 'COLONIA',
 'SECTOR',
 'CUADRANTE',
 'TURNO',
 'X',
 'Y']

Se pone en este nuevo orden el dataframe.

In [77]:
df = df[cols]
print(df)

          ID HORA_INICIO HORA_FIN      FECHA         MES  \
0          1       22:30    04:00   1/1/2016       ENERO   
1          2        6:40     6:40   1/1/2016       ENERO   
2          3       13:30    13:30   1/1/2016       ENERO   
3          4       03:00    05:00   1/1/2016       ENERO   
4          5       14:00    14:00   1/1/2016       ENERO   
5          6        0:05     0:05   1/1/2016       ENERO   
6          7        4:00     4:00   1/1/2016       ENERO   
7          8        4:58     4:58   1/1/2016       ENERO   
8          9       18:13    18:13   1/1/2016       ENERO   
9         10       19:40    19:40   1/1/2016       ENERO   
10        11       21:10    21:10   1/1/2016       ENERO   
11        12        7:20     7:20   1/1/2016       ENERO   
12        13       20:09    20:09   1/1/2016       ENERO   
13        14       15:34    15:34   1/1/2016       ENERO   
14        15       18:53    18:53   1/1/2016       ENERO   
15        16        2:40     2:40   1/1/

In [78]:
df.to_csv('InfoNeza/Delitos/Delitos_cleaned.csv')